In [11]:
%pip install langchain-community==0.2.16
%pip install langchain-core==0.2.39
%pip install langchain-experimental==0.0.64
%pip install langchain-huggingface==0.0.3
%pip install langchain-ollama==0.1.1
%pip install langchain-openai==0.1.23
%pip install langchain-qdrant==0.1.3
%pip install langchain-text-splitters==0.2.4
%pip install langdetect==1.0.9
%pip install langgraph==0.2.14
%pip install langgraph-checkpoint==1.0.6
%pip install langsmith==0.1.117
%pip install ragas==0.1.16

  Using cached langchain_core-0.2.41-py3-none-any.whl.metadata (6.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 71.8 MB/s eta 0:00:00
Using cached langchain_core-0.2.41-py3-none-any.whl (397 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.5
    Uninstalling langchain-core-0.3.5:
      Successfully uninstalled langchain-core-0.3.5
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.17
    Uninstalling langchain-community-0.2.17:
      Successfully uninstalled langchain-community-0.2.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.1.0 requires langchain-core<0.4,>=0.3.0, but you have langchain-core 0.2.41 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 25.9 MB/s eta 0:00:00
  Attempting uninsta

In [12]:
!pip install pymupdf

In [1]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

In [2]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [4]:

os.environ["HF_TOKEN"]=getpass.getpass("HF:")

In [6]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://download.pytorch.org/whl/cu117
     ---------------------------------------- 0.0/4.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/4.9 MB 1.3 MB/s eta 0:00:04
      --------------------------------------- 0.1/4.9 MB 919.0 kB/s eta 0:00:06
      --------------------------------------- 0.1/4.9 MB 1.2 MB/s eta 0:00:05
     - -------------------------------------- 0.2/4.9 MB 978.3 kB/s eta 0:00:05
     -- ------------------------------------- 0.3/4.9 MB 1.5 MB/s eta 0:00:04
     ---- ----------------------------------- 0.5/4.9 MB 2.2 MB/s eta 0:00:02
     ------- -------------------------------- 0.9/4.9 MB 3.1 MB/s eta 0:00:02
     --------- ------------------------------ 1.2/4.9 MB 3.9 MB/s eta 0:00:01
     ------------------ --------------------- 2.3/4.9 MB 6.2 MB/s eta 0:00:01
     -------------------- ------------------- 2.5/4.9 MB 6.6 MB/s eta 0:00:01
     ----------

  You can safely remove it manually.


In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "CamiloGC93/bge-large-en-v1.5-etical"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\juanm\anaconda3\envs\llms\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name CamiloGC93/bge-large-en-v1.5-etical. Creating a new one with mean pooling.


In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

# List of file paths for your PDFs
file_paths = ["dataset/data1.pdf", "dataset/data2.pdf"]  # Add more files here

# List to store all loaded documents
documents = []

# Loop through the file paths and load each PDF
for file_path in file_paths:
    loader = PyMuPDFLoader(file_path=file_path)
    docs = loader.load()  # Load documents from the current PDF
    documents.extend(docs)  # Add them to the overall documents list




In [ ]:
documents

# New test dataset SDG with the embeddiing finetuned


In [7]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = hf

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.25,
    reasoning: 0.25
}

In [8]:
testset = generator.generate_with_langchain_docs(documents, 15, distributions, with_debugging_logs=True)

Filename and doc_id are the same for all nodes.                   
Generating:   0%|          | 0/16 [00:00<?, ?it/s][ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 2, 'structure': 2, 'relevance': 3, 'score': 2.25}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['OSTP conducted meetings', 'Private sector and civil society stakeholders', 'Blueprint for an AI Bill of Rights', 'Positive use cases', 'Potential harms']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 3, 'structure': 2, 'relevance': 3, 'score': 2.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['GAI system incidents', 'Organizational risk management authority', 'Remediation plan', 'Deactivation criteria', 'Third-party GAI resources']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 1, 'relevance': 2, 'score': 1.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Racial equity', 'Supreme Court Decision',

In [9]:
dataset=testset.to_pandas()
dataset.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the focus of the Executive Order On Ad...,[ \nSECTION TITLE­\nFOREWORD\nAmong the great ...,The answer to given question is not present in...,simple,"[{'source': 'dataset/data1.pdf', 'file_path': ...",True
1,What are some examples of systems that impact ...,[APPENDIX\nSystems that impact the safety of c...,"Automated traffic control systems, electrical ...",simple,"[{'source': 'dataset/data1.pdf', 'file_path': ...",True
2,"How should designers, developers, and deployer...",[ \n \n \n \n \nSECTION TITLE\nDATA PRIVACY\nY...,"Designers, developers, and deployers of automa...",simple,"[{'source': 'dataset/data1.pdf', 'file_path': ...",True
3,What is the role of a purpose-built testing en...,[ \n31 \nMS-2.3-004 \nUtilize a purpose-built ...,A purpose-built testing environment like NIST ...,simple,"[{'source': 'dataset/data2.pdf', 'file_path': ...",True
4,What stakeholders were involved in providing i...,[APPENDIX\n• OSTP conducted meetings with a va...,Stakeholders involved in providing ideas relat...,simple,"[{'source': 'dataset/data1.pdf', 'file_path': ...",True


Evaluations chucks

RECURSIVE

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
rag_documents = text_splitter.split_documents(documents)

print(f"You have {len(rag_documents)} pages")

You have 465 pages


In [13]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="EticalAI"
)

In [14]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
client = QdrantClient(":memory:")

client.create_collection(
    collection_name="EticalAI",
    vectors_config=VectorParams(size=512 , distance=Distance.COSINE),
)

True

In [15]:
retriever = vectorstore.as_retriever()

In [17]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [18]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [28]:
rag_chain.invoke({"question" : "How do you monitor AI systems for potential misuse, such as generating false information or malicious content?"})

'To monitor AI systems for potential misuse, such as generating false information or malicious content, you can implement real-time monitoring processes for analyzing generated content performance and trustworthiness characteristics related to content provenance. This involves identifying deviations from desired standards and triggering alerts for human intervention. Additionally, it is recommended to engage in due diligence to analyze GAI output for harmful content, potential misinformation, and other related issues.'

### SemanticChunker

In [21]:
from langchain_experimental.text_splitter import SemanticChunker

In [24]:
semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
semantic_chunks = semantic_chunker.create_documents([d.page_content for d in documents])


In [25]:
vectorstore2 = Qdrant.from_documents(
    documents=semantic_chunks,
    embedding=embeddings,
    location=":memory:",
    collection_name="EticalAI2"
)

In [26]:
retriever2 = vectorstore2.as_retriever()

In [29]:
rag_chain2 = (
    {"context": itemgetter("question") | retriever2, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

#Recursive Chunking

In [31]:

rag_chain.invoke({"question" : "How do you monitor AI systems for potential misuse, such as generating false information or malicious content?"})

'To monitor AI systems for potential misuse, such as generating false information or malicious content, you can implement real-time monitoring processes that analyze generated content for performance and trustworthiness characteristics. This includes evaluating content provenance to identify deviations from desired standards and triggering alerts for human intervention. Additionally, engaging in due diligence to analyze AI output for harmful content and employing techniques like adversarial testing can help identify vulnerabilities and potential manipulation or misuse.'

Semantich Chunking

In [30]:
rag_chain2.invoke({"question" : "How do you monitor AI systems for potential misuse, such as generating false information or malicious content?"})

'To monitor AI systems for potential misuse, such as generating false information or malicious content, you can implement real-time monitoring processes for analyzing generated content performance and trustworthiness. This includes analyzing content provenance to identify deviations from desired standards and triggering alerts for human intervention. Additionally, you can compare AI system outputs against pre-defined organizational risk tolerance, guidelines, and principles, and review and test AI-generated content against these guidelines. Engaging in due diligence to analyze GAI output for harmful content and potential misinformation is also crucial.'

USING DATASET EVALUATION

In [32]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


#Recursive


In [33]:
retrieval_augmented_qa_chain_recursive = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | primary_qa_llm, "context": itemgetter("context")})


retrieval_augmented_qa_chain_semantic = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever2, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | primary_qa_llm, "context": itemgetter("context")})



In [35]:
test_questions = dataset["question"].values.tolist()
test_groundtruths = dataset["ground_truth"].values.tolist()

In [36]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain_recursive.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

In [40]:
answers_semantic = []
contexts_semantic = []

for question in test_questions:
  response = retrieval_augmented_qa_chain_recursive.invoke({"question" : question})
  answers_semantic.append(response["response"].content)
  contexts_semantic.append([context.page_content for context in response["context"]])

In [37]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [41]:
response_dataset_semantic = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers_semantic,
    "contexts" : contexts_semantic,
    "ground_truth" : test_groundtruths
})

In [38]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [39]:
results = evaluate(response_dataset, metrics)

Evaluating: 100%|██████████| 80/80 [00:58<00:00,  1.37it/s]


In [42]:
results_semantic = evaluate(response_dataset_semantic, metrics)

Evaluating: 100%|██████████| 80/80 [01:45<00:00,  1.32s/it]


In [45]:
recursive_results = results.to_pandas()
semantic_results=results_semantic.to_pandas()


In [53]:
semantic_results

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the focus of the Executive Order On Ad...,The focus of the Executive Order On Advancing ...,[managers-and-workers-are-losing-out\n105. Def...,The answer to given question is not present in...,1.000000,1.000000,1.000000,0.000000,0.174775
1,What are some examples of systems that impact ...,Some examples of systems that impact the safet...,[APPENDIX\nSystems that impact the safety of c...,"Automated traffic control systems, electrical ...",1.000000,0.948807,1.000000,1.000000,0.992631
2,"How should designers, developers, and deployer...","Designers, developers, and deployers of automa...",[SECTION TITLE\nDATA PRIVACY\nYou should be pr...,"Designers, developers, and deployers of automa...",1.000000,0.952726,0.500000,1.000000,0.822866
3,What is the role of a purpose-built testing en...,The role of a purpose-built testing environmen...,[31 \nMS-2.3-004 \nUtilize a purpose-built tes...,A purpose-built testing environment like NIST ...,0.714286,0.991522,1.000000,1.000000,0.694146
4,What stakeholders were involved in providing i...,The stakeholders involved in providing ideas r...,[ABOUT THIS FRAMEWORK­­­­­\nThe Blueprint for ...,Stakeholders involved in providing ideas relat...,0.750000,0.978718,0.000000,0.833333,0.235900
5,How have businesses successfully integrated AI...,Businesses have successfully integrated AI-dri...,[health coverage.”107\nThe customer service in...,Businesses have successfully integrated AI-dri...,1.000000,0.991692,1.000000,0.750000,0.998999
6,What funding opportunities are available for C...,The funding opportunities available for Cyber-...,[ENDNOTES\n23. National Science Foundation. Na...,Funding opportunities for Cyber-Physical Syste...,1.000000,1.000000,1.000000,1.000000,0.736314
7,What is the importance of building a glossary ...,I don't know.,[arXiv. https://arxiv.org/pdf/2308.14752 \nPar...,The importance of building a glossary for synt...,0.000000,0.000000,1.000000,1.000000,0.427830
8,How do structured feedback exercises help ensu...,Structured feedback exercises help ensure equi...,[29 \nMS-1.1-006 \nImplement continuous monito...,Structured feedback exercises help ensure equi...,0.833333,0.992425,1.000000,1.000000,0.993383
9,How to reduce national security risks in AI ri...,To reduce national security risks in AI risk m...,[and management. One possible way to further c...,"When managing AI risks, reducing national secu...",0.800000,0.982770,0.500000,1.000000,0.992898


In [46]:
recursive_results

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the focus of the Executive Order On Ad...,The focus of the Executive Order On Advancing ...,[managers-and-workers-are-losing-out\n105. Def...,The answer to given question is not present in...,1.000000,1.000000,1.000000,0.000000,0.926181
1,What are some examples of systems that impact ...,Some examples of systems that impact the safet...,[APPENDIX\nSystems that impact the safety of c...,"Automated traffic control systems, electrical ...",1.000000,0.948807,1.000000,1.000000,0.992631
2,"How should designers, developers, and deployer...","Designers, developers, and deployers of automa...",[SECTION TITLE\nDATA PRIVACY\nYou should be pr...,"Designers, developers, and deployers of automa...",1.000000,0.952726,0.666667,1.000000,0.822866
3,What is the role of a purpose-built testing en...,The role of a purpose-built testing environmen...,[31 \nMS-2.3-004 \nUtilize a purpose-built tes...,A purpose-built testing environment like NIST ...,0.714286,0.991522,1.000000,1.000000,0.594767
4,What stakeholders were involved in providing i...,The stakeholders involved in providing ideas r...,[ABOUT THIS FRAMEWORK­­­­­\nThe Blueprint for ...,Stakeholders involved in providing ideas relat...,0.750000,0.978721,0.000000,0.833333,0.235900
5,How have businesses successfully integrated AI...,Businesses have successfully integrated AI-dri...,[health coverage.”107\nThe customer service in...,Businesses have successfully integrated AI-dri...,1.000000,0.991692,1.000000,0.750000,0.998999
6,What funding opportunities are available for C...,The funding opportunities available for Cyber-...,[ENDNOTES\n23. National Science Foundation. Na...,Funding opportunities for Cyber-Physical Syste...,1.000000,1.000000,1.000000,1.000000,0.736314
7,What is the importance of building a glossary ...,I don't know.,[arXiv. https://arxiv.org/pdf/2308.14752 \nPar...,The importance of building a glossary for synt...,0.000000,0.000000,1.000000,1.000000,0.392115
8,How do structured feedback exercises help ensu...,Structured feedback exercises help ensure equi...,[29 \nMS-1.1-006 \nImplement continuous monito...,Structured feedback exercises help ensure equi...,1.000000,0.992448,1.000000,1.000000,0.718145
9,How to reduce national security risks in AI ri...,To reduce national security risks in AI risk m...,[and management. One possible way to further c...,"When managing AI risks, reducing national secu...",0.800000,0.982770,0.500000,1.000000,0.867895


In [47]:
# Select numerical columns
numerical_cols = recursive_results.select_dtypes(include=['number']).columns

# Create DataFrames with only numerical columns
df_num1 = recursive_results[numerical_cols]
df_num2 = semantic_results[numerical_cols]

In [48]:
# Reset indices to align DataFrames
df_num1 = df_num1.reset_index(drop=True)
df_num2 = df_num2.reset_index(drop=True)

# Ensure columns are in the same order
df_num2 = df_num2[df_num1.columns]


In [52]:
# Create a dictionary to hold comparison results
comparison_results = {}

for col in numerical_cols:
    # Compare the columns
    comparison = recursive_results[col] == semantic_results[col]
    
    # Identify rows with differences
    differences = recursive_results[~comparison]
    
    # Store the differences
    comparison_results[col] = differences[[col]].copy()
    
    # Optionally, include the differing values from both DataFrames
    comparison_results[col]['results_semantic'] = semantic_results.loc[~comparison, col].values

# Display the comparison results
for col, diff in comparison_results.items():
    if not diff.empty:
        print(f"\nDifferences in column: '{col}'")
        print(diff)
    else:
        print(f"\nNo differences found in column: '{col}'")


Differences in column: 'faithfulness'
    faithfulness  results_semantic
8       1.000000          0.833333
10      1.000000          0.875000
12      0.428571          0.461538
14      0.692308          0.769231

Differences in column: 'answer_relevancy'
    answer_relevancy  results_semantic
4           0.978721          0.978718
8           0.992448          0.992425
15          0.977774          0.977772

Differences in column: 'context_recall'
   context_recall  results_semantic
2        0.666667               0.5

Differences in column: 'context_precision'
   context_precision  results_semantic
1                1.0               1.0

Differences in column: 'answer_correctness'
    answer_correctness  results_semantic
0             0.926181          0.174775
2             0.822866          0.822866
3             0.594767          0.694146
7             0.392115          0.427830
8             0.718145          0.993383
9             0.867895          0.992898
10            0.8607

Based on the relevant metrics, the recursive chunk method is chosen. This choice is due to the fact that the semantic meaning used can be a drawback in this context, as it shows less reliability in more complex cases compared to the recursive method. The recursive is simpler computationally and, with adjustments to the chunk_size and overlap, could improve even further.

As for the embeddings method, the adjusted version shows better adaptation to the data when evaluating basic questions.